In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://salem:30333').getOrCreate()


2022-11-02 10:43:50,798 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
df = spark.read.parquet('/SO/posts.parquet')

In [5]:
df.select('_AcceptedAnswerId').distinct().count()

11755277